# BANA 4143 Final Project Part 1

## Step 1: Importing, Cleaning, & Preparing Data using Pandas

### Importing necessary packages & data

In [1]:
#Allows multiple outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Necessary packages
import pandas as pd       #Data cleaning and manipulation
import numpy as np        #Arithmetic and recoding data
import sqlalchemy as sqla #Interfacing with SQL Server database
import urllib             #Establishing connection with SQL Server database

In [3]:
####### MUST CHANGE FILE PATH FOR DIFFERENT USERS ########

#Define file path to the excel data file (vehicle_accidents.xlsx)
file_path = 'vehicle_accidents.xlsx'

#Read in data as a dataframe - only read in the 'accident_data' sheet - metadata sheet is not 
#necessary for our continued analysis
accidents_df = pd.read_excel(file_path, sheet_name = 'accident_data')

#Display data
accidents_df

,CRASH_DATE,CRASH_TIME,INJURY_RESULT,PERSON_AGE,INJURY_AREA,PERSON_SEX,INJURY_PERSON_TYPE
0,2022-11-16,06:00:00,Killed,80.0,Head,M,Pedestrian
1,2022-11-16,07:00:00,Injured,13.0,Back,F,Occupant
2,2022-11-16,03:00:00,Injured,37.0,Unknown,M,Occupant
3,2022-11-16,07:00:00,Injured,43.0,Chest,M,Occupant
4,2022-11-16,11:00:00,Injured,31.0,Knee-Lower Leg Foot,M,Bicyclist
...,...,...,...,...,...,...,...
45665,2022-01-01,12:00:00,Injured,25.0,Elbow-Lower-Arm-Hand,F,Pedestrian
45666,2022-01-01,00:00:00,Injured,26.0,Knee-Lower Leg Foot,M,Bicyclist
45667,2022-01-01,05:00:00,Injured,28.0,Head,F,Occupant
45668,2022-01-01,01:00:00,Injured,29.0,Back,M,Occupant


### Data Cleaning

In [4]:
#Display current data types
accidents_df.info()

#Recode datatypes to make them easier to work with 
#Doing this is especially necessary for 'CRASH_DATE' & 'CRASH_TIME' as we will concatenate them
accidents_df['CRASH_DATE'] = accidents_df['CRASH_DATE'].astype(str)
accidents_df['CRASH_TIME'] = accidents_df['CRASH_TIME'].astype(str)
accidents_df['INJURY_AREA'] = accidents_df['INJURY_AREA'].astype(str)
accidents_df['PERSON_SEX'] = accidents_df['PERSON_SEX'].astype(str)
accidents_df['INJURY_PERSON_TYPE'] = accidents_df['INJURY_PERSON_TYPE'].astype(str)

#Create datetime column from CRASH_DATE & CRASH_TIME
#This is accomplished by concatenating 'CRASH_DATE' & 'CRASH_TIME' and 
#then casting the concatenated string as a datetime data type
accidents_df['CRASH_DATETIME'] = pd.to_datetime(accidents_df['CRASH_DATE']+' '+ accidents_df['CRASH_TIME'])

#Drop 'CRASH_DATE' & 'CRASH_TIME' columns - no longer needed because of our created 'CRASH_DATETIME' column
accidents_df = accidents_df.drop(['CRASH_DATE','CRASH_TIME'], axis = 1)

#Display updated data frame
accidents_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45670 entries, 0 to 45669
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CRASH_DATE          45670 non-null  datetime64[ns]
 1   CRASH_TIME          45670 non-null  object        
 2   INJURY_RESULT       45670 non-null  object        
 3   PERSON_AGE          45250 non-null  float64       
 4   INJURY_AREA         45670 non-null  object        
 5   PERSON_SEX          45670 non-null  object        
 6   INJURY_PERSON_TYPE  45670 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 2.4+ MB


,INJURY_RESULT,PERSON_AGE,INJURY_AREA,PERSON_SEX,INJURY_PERSON_TYPE,CRASH_DATETIME
0,Killed,80.0,Head,M,Pedestrian,2022-11-16 06:00:00
1,Injured,13.0,Back,F,Occupant,2022-11-16 07:00:00
2,Injured,37.0,Unknown,M,Occupant,2022-11-16 03:00:00
3,Injured,43.0,Chest,M,Occupant,2022-11-16 07:00:00
4,Injured,31.0,Knee-Lower Leg Foot,M,Bicyclist,2022-11-16 11:00:00
...,...,...,...,...,...,...
45665,Injured,25.0,Elbow-Lower-Arm-Hand,F,Pedestrian,2022-01-01 12:00:00
45666,Injured,26.0,Knee-Lower Leg Foot,M,Bicyclist,2022-01-01 00:00:00
45667,Injured,28.0,Head,F,Occupant,2022-01-01 05:00:00
45668,Injured,29.0,Back,M,Occupant,2022-01-01 01:00:00


In [5]:
#accidents_df['PERSON_AGE'].replace([-971,-967,-966,-964,-961,-959,-956,-951,-943,-797,-594,-593,-592,112,120,121,
# 150,848,999,1050,1055,1981,4043,4030,5210,9999],np.nan)

#Recode age values greater than 100 years old or less than 0 to be Not a Number (essentially NA)
accidents_df.loc[(accidents_df['PERSON_AGE'] < 0) | (accidents_df['PERSON_AGE'] > 100), 'PERSON_AGE'] = np.NaN

#Fill NaN values with -999 to represent missing data
accidents_df['PERSON_AGE'] = accidents_df['PERSON_AGE'].fillna(-999)

#Cast column to int data type - was previously a float
accidents_df['PERSON_AGE'] = accidents_df['PERSON_AGE'].astype(int)

#Display updated table
accidents_df

,INJURY_RESULT,PERSON_AGE,INJURY_AREA,PERSON_SEX,INJURY_PERSON_TYPE,CRASH_DATETIME
0,Killed,80,Head,M,Pedestrian,2022-11-16 06:00:00
1,Injured,13,Back,F,Occupant,2022-11-16 07:00:00
2,Injured,37,Unknown,M,Occupant,2022-11-16 03:00:00
3,Injured,43,Chest,M,Occupant,2022-11-16 07:00:00
4,Injured,31,Knee-Lower Leg Foot,M,Bicyclist,2022-11-16 11:00:00
...,...,...,...,...,...,...
45665,Injured,25,Elbow-Lower-Arm-Hand,F,Pedestrian,2022-01-01 12:00:00
45666,Injured,26,Knee-Lower Leg Foot,M,Bicyclist,2022-01-01 00:00:00
45667,Injured,28,Head,F,Occupant,2022-01-01 05:00:00
45668,Injured,29,Back,M,Occupant,2022-01-01 01:00:00


### Data Preparation (creating dataframes based off of ERD)

In [6]:
####################################### Creating Crash Table ########################################################

#Create an empty dataframe object
#This allows us to name the dataframe and begin adding in our columns
crash_df = pd.DataFrame()

#Populate columns - the only column we need is the datetime of the crash
#Each crash is defined by a unique Datetime 
crash_df['crash_dttm'] = accidents_df['CRASH_DATETIME'].unique()

#Index will serve as the 'crash_id' a unique identifier for each crash datetime
#Make index start from 1 (indices begin at 0)
crash_df.index += 1

#Add 'C' to the index number to distinguish the 'crash_id'
crash_df.index = 'C' + crash_df.index.astype(str)

#Rename index to 'crash_id'
crash_df.index.name = 'crash_id'

#Reset index to change 'crash_id' into a column 
#This will allow us to merge other dataframes based on this field
crash_df = crash_df.reset_index()

#Display Crash Table
crash_df

,crash_id,crash_dttm
0,C1,2022-11-16 06:00:00
1,C2,2022-11-16 07:00:00
2,C3,2022-11-16 03:00:00
3,C4,2022-11-16 11:00:00
4,C5,2022-11-16 15:00:00
...,...,...
7095,C7096,2022-01-01 22:00:00
7096,C7097,2022-01-01 21:00:00
7097,C7098,2022-01-01 14:00:00
7098,C7099,2022-01-01 23:00:00


In [7]:
###################### Creating Demographics/People Table ################################################

#Merge master table (accidents_df) with crash table (crash_df) in order to capture 'crash_id' & all fields
#'crash_id' will be used as a foreign key in this table
demo_merge_df = pd.merge(accidents_df, crash_df, left_on = "CRASH_DATETIME", right_on = "crash_dttm")

#Create a new DataFrame for demographic information containing only the necessary fields
demographic_df = demo_merge_df[['crash_id','PERSON_AGE', 'PERSON_SEX', 'INJURY_PERSON_TYPE']]

#Reset index
demographic_df.reset_index(drop=True, inplace=True)
#Begin index at 1
demographic_df.index += 1
##Add 'P' to the index number to distinguish the 'person_id'
demographic_df.index = 'P' + demographic_df.index.astype(str)
#Rename index to 'person_id'
demographic_df.index.name = 'person_id'

#Reset index to change 'person_id' into a column 
#Doing this will make inserting the 'person_id' into SQL tables possible
demographic_df = demographic_df.reset_index()

#Display Demographics/People
demographic_df

,person_id,crash_id,PERSON_AGE,PERSON_SEX,INJURY_PERSON_TYPE
0,P1,C1,80,M,Pedestrian
1,P2,C1,41,F,Occupant
2,P3,C1,49,M,Occupant
3,P4,C2,13,F,Occupant
4,P5,C2,43,M,Occupant
...,...,...,...,...,...
45665,P45666,C7098,21,M,Occupant
45666,P45667,C7099,17,M,Bicyclist
45667,P45668,C7099,20,M,Occupant
45668,P45669,C7100,78,F,Occupant


In [8]:
########################################## Create Injury Table #######################################################

####### Step 1: Obtain table containing all fields from the original file, as well as 'crash_id', & 'person_id' ######

#Merge master table (accidents_df) with crash table (crash_df) in order to capture 'crash_id' & all fields
injury_merge_df = pd.merge(accidents_df, crash_df, left_on = "CRASH_DATETIME", right_on = "crash_dttm")

#Reset and drop existing index 
injury_merge_df.reset_index(drop=True, inplace=True)

#Begin new index at 1
injury_merge_df.index += 1
#Add 'P' to the index number to distinguish the 'person_id'
injury_merge_df.index = 'P' + injury_merge_df.index.astype(str)
#Rename index to 'person_id'
injury_merge_df.index.name = 'person_id'

#Reset index to change 'person_id' into a column in our new dataframe (injury_merge_df)
injury_merge_df = injury_merge_df.reset_index()

#Display our merged table
#Merged table contains all fields from the original file (vehicle_accidents) and 'crash_id' and 'person_id'
injury_merge_df

,person_id,INJURY_RESULT,PERSON_AGE,INJURY_AREA,PERSON_SEX,INJURY_PERSON_TYPE,CRASH_DATETIME,crash_id,crash_dttm
0,P1,Killed,80,Head,M,Pedestrian,2022-11-16 06:00:00,C1,2022-11-16 06:00:00
1,P2,Injured,41,Head,F,Occupant,2022-11-16 06:00:00,C1,2022-11-16 06:00:00
2,P3,Injured,49,Entire Body,M,Occupant,2022-11-16 06:00:00,C1,2022-11-16 06:00:00
3,P4,Injured,13,Back,F,Occupant,2022-11-16 07:00:00,C2,2022-11-16 07:00:00
4,P5,Injured,43,Chest,M,Occupant,2022-11-16 07:00:00,C2,2022-11-16 07:00:00
...,...,...,...,...,...,...,...,...,...
45665,P45666,Injured,21,Back,M,Occupant,2022-01-01 14:00:00,C7098,2022-01-01 14:00:00
45666,P45667,Injured,17,Chest,M,Bicyclist,2022-01-01 23:00:00,C7099,2022-01-01 23:00:00
45667,P45668,Injured,20,Back,M,Occupant,2022-01-01 23:00:00,C7099,2022-01-01 23:00:00
45668,P45669,Injured,78,Head,F,Occupant,2022-01-01 09:00:00,C7100,2022-01-01 09:00:00


In [9]:
########################################## Create Injury Table #######################################################

##########  Step 2: Split injury field to contain a unique record for each afflicted injury area  ###################

#Create a new dataframe from a copy of the merged dataframe created above
#This dataframe will act as our Injury Table
injury_df = injury_merge_df.copy()

#Recode injury areas to use '-' as the delimiter between injury areas
#Recoding by repoisition the delimiter will allow us to split the field into unique records for each injured area
#Recoding 'Does Not Apply' & 'Unknown' to NA values will allow us to drop those records from the table
#(Because this table is meant only to house injuries, it wouldn't make sense to record an entry claiming no injury exists)
injury_df['INJURY_AREA'] = injury_df['INJURY_AREA'].replace(['Elbow-Lower-Arm-Hand','Knee-Lower Leg Foot','Does Not Apply','Unknown'],
                                                                  ['Elbow-Lower Arm-Hand', 'Knee-Lower Leg-Foot', pd.NA, pd.NA])

#Split injury areas based on '-' delimiter
injury_df['INJURY_AREA'] = injury_df['INJURY_AREA'].str.split('-')

#Make each split value into its own record
#Each injury area will now have its own row in the dataframe 
injury_df = injury_df.explode('INJURY_AREA')

#Trim trailing or leading spaces from the split values
injury_df['INJURY_AREA'] = injury_df['INJURY_AREA'].str.strip()

#Remove null records from the 'INJURY_AREA' column 
#This ensures we only have records in our table for injuries that occurred
injury_df = injury_df.dropna(subset=['INJURY_AREA'])

#Recode 'INJURY_RESULT' values to describe severity - this is not necessary, just preference
injury_df['INJURY_RESULT'] = injury_df['INJURY_RESULT'].replace(['Killed','Injured'],['Fatal','Non-Fatal'])

In [10]:
########################################## Create Injury Table #######################################################

####################  Step 3: Create 'injury_id' field & filter out unneccessary data #############################

#Update the table to only show the necessary fields
injury_df = injury_df[['person_id','INJURY_AREA','INJURY_RESULT']]

#Reset and drop existing index 
injury_df.reset_index(drop=True, inplace=True)
#Begin new index at 1
injury_df.index += 1
#Add 'I' to the index number to distinguish the 'injury_id'
injury_df.index = 'I' + injury_df.index.astype(str)
#Change index name to 'injury_id'
injury_df.index.name = 'injury_id'
#Reset index to change 'injury_id' into a column 
#Doing this will make inserting the 'injury_id' into SQL tables possible
injury_df = injury_df.reset_index()

#Display Injury Table
injury_df

,injury_id,person_id,INJURY_AREA,INJURY_RESULT
0,I1,P1,Head,Fatal
1,I2,P2,Head,Non-Fatal
2,I3,P3,Entire Body,Non-Fatal
3,I4,P4,Back,Non-Fatal
4,I5,P5,Chest,Non-Fatal
...,...,...,...,...
71365,I71366,P45666,Back,Non-Fatal
71366,I71367,P45667,Chest,Non-Fatal
71367,I71368,P45668,Back,Non-Fatal
71368,I71369,P45669,Head,Non-Fatal


In [11]:
########################################## Create Injury Table #######################################################

####################  Step 4: Create 'InjuryAreaID' field & filter out unneccessary data #############################

#Create keys corresponding to each injury area
#A dictionary is being used to declare key-value pairs which will allow us to map these into the existing injury_df
injury_area_dict = {'Back': 'A1',
                    'Lower Leg': 'A2',
                    'Foot': 'A3',
                    'Knee': 'A4',
                    'Neck': 'A5',
                    'Head': 'A6',
                    'Entire Body': 'A7',
                    'Hand': 'A8',
                    'Elbow': 'A9',
                    'Lower Arm': 'A10',
                    'Shoulder': 'A11',
                    'Upper Arm': 'A12',
                    'Chest': 'A13',
                    'Upper Leg': 'A14',
                    'Hip': 'A15',
                    'Face': 'A16',
                    'Pelvis': 'A17',
                    'Abdomen': 'A18',
                    'Eye': 'A19'}

#Map key values to the injury area column and store the keys in another column named 'InjuryAreaID'
#The map function takes the key values assigned to each injury area in the dictionary above
#and assigns the corresponding key to each instance of an injury area
#A new column 'InjuryAreaID' was created to store these values
injury_df["InjuryAreaID"] = injury_df['INJURY_AREA'].map(injury_area_dict)

#Update the table to only show the necessary fields
#Now that we have an 'InjuryAreaID' column, we no longer need to show the injury area as a column
injury_df = injury_df[['injury_id','person_id','InjuryAreaID','INJURY_RESULT']]

#Display updated and finalized injury table
injury_df

,injury_id,person_id,InjuryAreaID,INJURY_RESULT
0,I1,P1,A6,Fatal
1,I2,P2,A6,Non-Fatal
2,I3,P3,A7,Non-Fatal
3,I4,P4,A1,Non-Fatal
4,I5,P5,A13,Non-Fatal
...,...,...,...,...
71365,I71366,P45666,A1,Non-Fatal
71366,I71367,P45667,A13,Non-Fatal
71367,I71368,P45668,A1,Non-Fatal
71368,I71369,P45669,A6,Non-Fatal


In [12]:
########################################## Create InjuryArea Table #######################################################

#Create a dataframe from the injury_area_dict dictionary above
#The columns argument renames the columns to the dataframe to match what is being stored in the columns
injury_area_df = pd.DataFrame(injury_area_dict.items(), columns=['InjuryArea', 'InjuryAreaID'])

#Update the data frame to show 'InjuryAreaCode' before 'InjuryArea'
#Essentially we are just flipping the order the columns are being displayed
injury_area_df = injury_area_df[['InjuryAreaID','InjuryArea']]

#Display InjuryArea Table
injury_area_df

,InjuryAreaID,InjuryArea
0,A1,Back
1,A2,Lower Leg
2,A3,Foot
3,A4,Knee
4,A5,Neck
5,A6,Head
6,A7,Entire Body
7,A8,Hand
8,A9,Elbow
9,A10,Lower Arm


## Step 2: Creating & Implementing Tables in SQL using DDL & DML

### Connect to SQL Server

In [13]:
#Define connection details

####################### MUST UPDATE DATABASE, UID, AND PWD TO CONNECT TO YOUR OWN DATABASE ####################
conn_details = urllib.parse.quote_plus("""DRIVER={ODBC Driver 17 for SQL Server};
                                       SERVER=lcb-sql.uccob.uc.edu\DMFA;
                                       DATABASE=LCB-DMFA-005;
                                       UID=LCB-DMFA-005;PWD=PZ$022bd""")

#Create database engine using connection details
db_engine = sqla.create_engine("mssql+pyodbc:///?odbc_connect=%s" % conn_details)

#Connect to server
conn2 = db_engine.connect()

### Create tables in SQL Server (DDL)

In [41]:
#A schema named 'final' is created to house our tables
#While this is not an absolutely necessary step, it helps organize our database 
#and is generally considered best practice

#Create 'final' schema for tables
try:
    conn2.execute(sqla.schema.CreateSchema('final'))
except Exception as e:
    print(e)

(pyodbc.ProgrammingError) ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'final' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]CREATE SCHEMA failed due to previous errors. (2759)")
[SQL: CREATE SCHEMA final]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [24]:
########################################## Create SQL Crashes Table ################################################

#The Crashes table contains just two fields - CrashID & CrashDateTime
#CrashID serves as the primary key for the table
#CrashID is not at first declared as the primary key in order to allow for a primary key constraint to be added
#For this reason, CrashID is first declared as having to be UNIQUE and NOT NULL 
#(two defining characteristics of a primary key)
#CrashID is given the datatype VARCHAR as it contains a combination of letters and numbers (i.e. "C1")
#We refrained from simply using an IDENTITY to define the primary key to allow for the key to easily be distinguished
#and differentiated from the keys of other tables
#CrashDateTime is given the datatype DATETIME as it contains a combined date and time in the field

#Create text object containing SQL DDL
crash_tbl_cmd = sqla.text(
    """CREATE TABLE final.Crashes (
    CrashID VARCHAR(10) UNIQUE NOT NULL,
    CrashDateTime DATETIME NOT NULL,
    CONSTRAINT CrashPK PRIMARY KEY (CrashID)
    );""")

#Execute SQL command above
try:
    conn2.execute(crash_tbl_cmd)
except Exception as e:
    print(e)

In [25]:
########################################## Create SQL People Table ################################################

#The People table contains five fields - PersonID, CrashID, Age, Sex, & Type
#PersonID serves as the primary key for the table
#Similar to the crash table, a primary key constraint is placed on PersonID and the field is initially
#declared to be UNIQUE and NOT NULL (two defining characteristics of a primary key)
#PersonID is given the datatype VARCHAR as it contains a combination of letters and numbers (i.e. "P1")
#CrashID is given a foreign key constraint referencing the Crashes table we just created
#This allows for us to easily join the two tables
#CrashID has the same datatype as it does in the Crashes table; 
#however, it is not declared UNIQUE as there may exist several instances of it in the table
#Age is given the datatype INT
#Sex is given the datatype CHAR and limited to just 1 character
#Additionally, a constraint has been added to ensure that the Sex entered 
#may only be F for Female, M for male, or U for Unknown
#Type is given the datatype VARCHAR to allow for varying length strings to be entered identifying the type of person
#involved in a crash
#All columns created for the table do not accept null values - denoted by the NOT NULL arguments found following 
#the corresponding data type declarations
#This ensures that data must be entered for each field, prioritizing data integrity

#Create text object containing SQL DDL
people_tbl_cmd = sqla.text(
    """CREATE TABLE final.People (
    PersonID VARCHAR(10) UNIQUE NOT NULL,
    CrashID VARCHAR(10) NOT NULL,
    Age INT NOT NULL,
    Sex CHAR(1) NOT NULL,
    Type VARCHAR (20) NOT NULL,
    CONSTRAINT PersonSexCheck CHECK (Sex IN ('F','M','U')),
    CONSTRAINT PersonPK PRIMARY KEY (PersonID),
    CONSTRAINT CrashFK FOREIGN KEY (CrashID) REFERENCES final.Crashes(CrashID)
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
    );""")
#Added constraint to ensure Sex is entered properly

#Execute SQL command above
try:
    conn2.execute(people_tbl_cmd)
except Exception as e:
    print(e)

In [18]:
########################################## Create SQL InjuryAreas Table ################################################

#The InjuryAreas table contains two fields - InjuryAreaID & InjuryArea
#InjuryAreaID serves as the primary key for the table
#Similar to the crash and people table, a primary key constraint is placed on InjuryAreaID and the field is initially
#declared to be UNIQUE and NOT NULL (two defining characteristics of a primary key)
#InjuryAreaID is given the datatype VARCHAR as it contains a combination of letters and numbers (i.e. "A1")
#This field is limited to just 3 characters as only 19 injury areas exist
#There are no foreign keys in this table as it is strictly meant to house the 
#unique injury areas associated with each injury
#InjuryArea is given the data type VARCHAR and allowed 20 characters to accommodate strings of varying length

#Create text object containing SQL DDL
injury_area_tbl_cmd = sqla.text(
    """CREATE TABLE final.InjuryAreas (
    InjuryAreaID VARCHAR(3) UNIQUE NOT NULL,
    InjuryArea VARCHAR(20) NOT NULL,
    CONSTRAINT InjuryAreaPK PRIMARY KEY (InjuryAreaID)
    );""")

#Execute SQL command above
try:
    conn2.execute(injury_area_tbl_cmd)
except Exception as e:
    print(e)

In [19]:
########################################## Create SQL Injuries Table ################################################

#The Injuries table contains four fields - InjuryID, PersonID, InjuryAreaID, and Severity
#InjuryID serves as the primary key for the table
#Similar to the tables created above, a primary key constraint is placed on InjuryAreaID and the field is initially
#declared to be UNIQUE and NOT NULL (two defining characteristics of a primary key)
#InjuryID is given the datatype VARCHAR as it contains a combination of letters and numbers (i.e. "A1")
#The data types for PersonID and InjuryAreaID match those found in their corresponding tables
#Both PersonID and InjuryAreaID are given foreign key constraints that reference the table with which they
#act as primary keys - this allows for the joining of data
#Severity is given the data type VARCHAR and allowed 20 characters to accommodate strings of varying length
#NULL values are not allowed for any field in this table as it ensures data integrity and the ability to join data
#(for instance, if PersonID were allowed to be NULL, it would be impossible to match certain records back to the 
#People table and subsequently the Crashes table)

#Create text object containing SQL DDL
injuries_tbl_cmd = sqla.text(
    """CREATE TABLE final.Injuries (
    InjuryID VARCHAR(10) UNIQUE NOT NULL,
    PersonID VARCHAR(10) NOT NULL,
    InjuryAreaID VARCHAR(3) NOT NULL,
    Severity VARCHAR(20) NOT NULL,
    CONSTRAINT InjuryPK PRIMARY KEY (InjuryID),
    CONSTRAINT PersonFK FOREIGN KEY (PersonID) REFERENCES final.People(PersonID)
        ON UPDATE NO ACTION
        ON DELETE NO ACTION,
    CONSTRAINT InjuryAreaFK FOREIGN KEY (InjuryAreaID) REFERENCES final.InjuryAreas(InjuryAreaID)
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
    );""")

#Execute SQL command above
try:
    conn2.execute(injuries_tbl_cmd)
except Exception as e:
    print(e)

### Insert information into SQL tables (DML)

In [28]:
########################################## Insert Into Crash Table ##################################################
#Create insert dataframe by copying crash_df
crashes_insert_df = crash_df.copy()

#Rename columns to match database table
crashes_insert_df = crashes_insert_df.rename(columns={'crash_id': 'CrashID', 'crash_dttm': 'CrashDateTime'})

#Insert crashes_insert_df into final.Crashes table in SQL
try:
    crashes_insert_df.to_sql('Crashes',conn2,schema='final',if_exists='append',index=False, chunksize=1000, method=None)
except Exception as e:
    print(e)

-1

In [29]:
########################################## Insert Into People Table ##################################################
#Create insert dataframe by copying demographic_df
people_insert_df = demographic_df.copy()

#Rename columns to match database table
people_insert_df = people_insert_df.rename(columns={'person_id':'PersonID', 'crash_id':'CrashID',
                                                    'PERSON_AGE':'Age', 'PERSON_SEX':'Sex', 
                                                    'INJURY_PERSON_TYPE':'Type'})

#Insert people_insert_df into final.People table in SQL
try:
    people_insert_df.to_sql('People',conn2,schema='final',if_exists='append',index=False, chunksize=1000, method=None)
except Exception as e:
    print(e)


-1

In [20]:
########################################## Insert Into InjuryAreas Table ##################################################
#Create insert dataframe by copying injury_df
injury_area_insert_df = injury_area_df.copy()

#Insert injuries_insert_df into final.Injuries table in SQL
try:
    injury_area_insert_df.to_sql('InjuryAreas',conn2,schema='final',if_exists='append',index=False)
except Exception as e:
    print(e)

-1

In [14]:
########################################## Insert Into Injuries Table ##################################################
#Create insert dataframe by copying injury_df
injuries_insert_df = injury_df.copy()

#Rename columns to match database table
injuries_insert_df = injuries_insert_df.rename(columns={'injury_id':'InjuryID', 'person_id':'PersonID', 
                                                                 'InjuryAreaID':'InjuryAreaID','INJURY_RESULT':'Severity'})

#Insert injuries_insert_df into final.Injuries table in SQL
try:
    injuries_insert_df.to_sql('Injuries',conn2,schema='final',if_exists='append',index=False, chunksize=1000, method=None)
except Exception as e:
    print(e)

-72

### Close database connection

In [ ]:
#Close connection
conn2.close() 

#Close engine
db_engine.dispose() 